# Projet 4 Bernadoy Corentin

Le but de ce projet est de créer une balise html `<a>` autour des textes de référencés dans les différents arrêtés municipaux mis à notre disposition.

## Imports

In [104]:
import os
import bs4
import regex as re

## Stratégie et étude du jeu de donnée

Pour une première approche, nous allons tenter d'utiliser un maximum les expressions régulières. En effet, il s'agit d'une approche simple et si elle réussit, permet de consitituer un premier jeu de données labélisées pour une étude par machine learning. 

Tout d'abord on établit la structure que l'on veut suivre : on va extraire les textes présents dans les balises via Beautifulsoup, recherche la présence de textes de lois dans ces textes extraits, les encadrer par une balise `<a>` puis on va sauvegarder le fichier html modifié dans un fichier séparé.

### Création des dossier de dépôt des résultats

On commence par créer les dossier de résultats dans l'arborescence actuelle

In [105]:
#cellule pour créer les dossiers de résultats
root = "./data/"
os.makedirs(root + "results/", exist_ok= True)
sub_data_dirs = next(os.walk(root + 'data/'))[1]
for sub_data_dir in sub_data_dirs:
    os.makedirs(root + "results/" + sub_data_dir, exist_ok= True)

### Fonctions utiles

Il nous faut trois fonctions pour établir la structure : une première fonction `find_regex` pour évaluer les matchs avec les regex et déterminer leur position dans le texte, une fonction `rewritting` pour réécrire le texte avec les balises et enfin une fonction  `pipeline` qui itère les deux premières sur chaque texte de chaque fichier et qui sauvegarde les résultats

On écrit d'abord la list des regex que l'on va utiliser pour détecter les références à des textes de lois (plus de détails sur comment établir les regexs dans la partie suivante)

In [106]:
regex_list = [
    r"directive[\sA-Za-zÀ-ÿ]+\d{4}/\d+/[A-Z]{2}",
    r"[Dd]écret(?:\s+n°|\s+no)?\s*[\d\-/.]+(?:\s+du\s+\d{1,2}(?:er)?\s+[a-zéû]+\s+\d{4})?|[Dd]écret\s+du\s+\d{1,2}(?:er)?\s+[a-zéû]+\s+\d{4}",
    r"article(.+?)[Cc]ode\s+de\s+l[’']environnement|[\sA-Za-zÀ-ÿ]{2}\s+[Cc]ode\s+de\s+l[’']environnement",
    r"[Aa]rrêté\s[Pp]réfectora[lux]+(?:\s+n°|\s+no)\s*[\d\-]+|[Aa]rrêté[s\s]+[Pp]réfectora[lux]+du\s+\d{1,2}(?:er)?\s+[a-zéû]+\s+\d{4}",
    r"\s[Ll]oi(?:\s+n°|\s+no)?\s*[\d\-\s]+(?:du\s+\d{1,2}(?:er)?\s+[a-zéû]+\s+\d{4})?",
    r"circulaire[\sA-Za-zÀ-ÿ]+(?:\s+n°|\s+no)(?:[A-Z/\d]+)\s+(?:du\s+\d{1,2}(?:er)?\s+[a-zéû]+\s+\d{4})?|circulaire[\sA-Za-zÀ-ÿ]+(?:du\s+\d{1,2}(?:er)?\s+[a-zéû]+\s+\d{4})",
]

In [107]:
cde = regex_list[-1]

In [108]:
### only for tests (if somebody is curious enough to look until here in the commits :))

with open('data/data/0022200009/1999-12-21_AP-auto_pixtral.html', 'r') as f:
    soup = bs4.BeautifulSoup(f, 'html')   
    whole_text = soup.find_all(['div', 'td', 'h2'])
    for tag in whole_text:
        found, occurences = find_regex(regex_list, tag.text)
        if found:
            sorted_occurence = sorted(occurences)
            spliced_text = []
            start = 0
            for occ in sorted_occurence:
                spliced_text.append(tag.text[start:occ[0]])
                spliced_text.append(tag.text[occ[0]:occ[1]])
                start = occ[1]
            spliced_text.append(tag.text[start:])
            tag.string = ''
            for i, substring in enumerate(spliced_text):
                if (i%2 == 0):
                    tag.append(substring)
                if (i%2 != 0):
                    a_tag = soup.new_tag('a')
                    a_tag.string = substring
                    tag.append(a_tag)
                
                



On écrit les fonctions

In [109]:
def find_regex(regex_list, text):
    occurences = []
    for regex in regex_list:
        for occurence in re.finditer(regex, text):
            occurences.append((occurence.start(0),occurence.end(0)))
    return((len(occurences) > 0), occurences)

In [110]:
def rewriting(soup, tag, occurences):
    sorted_occurence = sorted(occurences)
    spliced_text = []
    start = 0
    for occ in sorted_occurence:
        spliced_text.append(tag.text[start:occ[0]])
        spliced_text.append(tag.text[occ[0]:occ[1]])
        start = occ[1]
    spliced_text.append(tag.text[start:])
    tag.string = ''
    for i, substring in enumerate(spliced_text):
        if (i%2 == 0):
            tag.append(substring)
        if (i%2 != 0):
            a_tag = soup.new_tag('a')
            a_tag.string = substring
            tag.append(a_tag)

In [111]:
root = "./data/"
def pipeline(root = root):
    data_dirs = root + 'data/'
    sub_data_dirs = next(os.walk(data_dirs))[1]
    res_dirs = root + 'results/'
    for sub_data_dir in sub_data_dirs:
        file_names = next(os.walk(data_dirs + sub_data_dir + '/'))[2]
        for file_name in file_names:
            data_path = data_dirs + sub_data_dir + '/' + file_name
            res_data_path = res_dirs + sub_data_dir + '/' + file_name[:-5] + '_results.html'
            print(data_path) # à retirer à la fin : uniquement pour du troubleshooting
            with open(data_path, 'r') as f:
                soup = bs4.BeautifulSoup(f, 'html')   
                whole_text = soup.find_all(['div', 'td', 'h2'])
                for tag in whole_text:
                    if (not(tag.find('h1'))): # on exclue le div qui contient le titre, car ce dernier contient toujours le nom du texte présenté dans le document, ce qui n'est pas une référence externe
                        found, occurences = find_regex(regex_list, tag.text)
                        if found:
                            rewriting(soup, tag, occurences) 

            with open(res_data_path, 'w', encoding= "utf-8") as f:
                    f.write(str(soup))

In [112]:
pipeline()

./data/data/0003013459/2020-04-20_AP-auto_initial_pixtral.html
./data/data/0003013459/2021-09-24_AP-auto_refonte_pixtral.html
./data/data/0003013459/2023-02-23_APC-auto_pixtral.html
./data/data/0005205103/2002-12-11_AP-auto_initial_pixtral.html
./data/data/0005205103/2004-04-16_AP-auto_refonte_pixtral.html
./data/data/0005205103/2004-05-04_APC-auto_pixtral.html
./data/data/0005205103/2004-05-04_APC-TAR_pixtral.html
./data/data/0005205103/2006-01-31_AP-audit_pixtral.html
./data/data/0005205103/2007-10-25_AP-auto_refonte_pixtral.html
./data/data/0005205103/2008-08-14_AP-radioactif_pixtral.html
./data/data/0005205103/2008-11-26_AP-audit_pixtral.html
./data/data/0005205103/2009-09-11_AP-auto_refonte_pixtral.html
./data/data/0005205103/2010-04-02_AP-MU_pixtral.html
./data/data/0005205103/2011-07-18_AP-RSDE_pixtral.html
./data/data/0005205103/2012-01-11_AP-auto_refonte_pixtral.html
./data/data/0005205103/2012-01-11_AP-auto_refonte_pixtral_old.html
./data/data/0005205103/2013-10-23_AP-auto_re

## Comment établir les regexs :

On écrit une fonction `get_example` qui permet de récupérer toutes les balises contenant un mot de texte de loi à référencer, ce qui nous permet de récupérer toutes les occurences d'un type de loi dans les documents et donc d'avoir une idée de quelle regexp utiliser. On sauvegarde ces textes dans un fichier text séparé afin de pouvoir l'étudier plus facilement.

In [113]:
#### ne pas supprimer : permet de savoir quel format peut avoir chaque texte de loi
def get_examples(Loi, name):
    root = "./data/"
    data_dirs = root + 'data/'
    sub_data_dirs = next(os.walk(data_dirs))[1]
    res_dirs = root + 'results/'
    examples = []
    for sub_data_dir in sub_data_dirs:
        file_names = next(os.walk(data_dirs + sub_data_dir + '/'))[2]
        for file_name in file_names:
            data_path = data_dirs + sub_data_dir + '/' + file_name
            res_data_path = res_dirs + sub_data_dir + '/' + file_name[:-5] + '_results.html'
            with open(data_path, 'r') as file_content:
                soup = bs4.BeautifulSoup(file_content, 'html')   
                whole_text = soup.find_all(['div', 'td', 'h2'])
                for tag in whole_text:
                    if(find_regex(Loi, tag.text)[0]):
                        examples.append(tag.text)

    with open(f"example_{name}.txt", 'w') as f:
        for example in examples:
            f.write(f"{example}\n")

In [114]:
get_examples([r"[Cc]irculaire"], "circulaire")